# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from dotenv import load_dotenv
import pandas as pd

import data_import
from enrich import process_transactions
import model

load_dotenv("../user.env")

True

# Fetch


In [3]:
import ast
import os
from loguru import logger

from pathlib import Path

from services.nordigen import config, account_api
import utils

In [4]:
def get_requisition_id():
    requisition_ids = ast.literal_eval(os.environ.get("requisition_ids"))
    logger.info(f"found {len(requisition_ids)} requisition_ids")

    if not requisition_ids:
        raise ValueError("no requisition ids found")

    if not isinstance(requisition_ids, list):
        raise TypeError("invalid type found")

    if len(requisition_ids) > 1:
        logger.warning("More than 1 requisition id found")
        raise NotImplementedError("unhandled >1 requisition")

    requisition_id = requisition_ids[0]
    return requisition_id


# get first requisition id


output_filepath = await account_api.get_latest_data(requisition_id=get_requisition_id())

2024-01-18 13:21:51.287 | INFO     | __main__:get_requisition_id:3 - found 1 requisition_ids
2024-01-18 13:21:51.288 | INFO     | services.nordigen.account_api:get_latest_data:226 - today's data exists


In [6]:
latest_data = data_import.load_user_data(filepath=output_filepath)

In [9]:
pd.DataFrame(latest_data.transactions)

,booking_date,booking_date_time,remittance_information_unstructured,proprietary_bank_transaction_code,amount,transaction_currency,status,transaction_id,internal_transaction_id,account_id,...,target_currency,quotation_date,value_date,value_date_time,booking_year,booking_month,flow,counterparty,category,year_month
0,2024-01-16,2024-01-16,"BANK GIRO CREDIT REF SATALIA NP COMPLET, .",BANK TRANSFER CREDIT,186.20,None,booked,e770cf3d-b34c-32b7-acbd-2dbe7f94cdcd,cfd6a5cecb851cd6f88f259dcfba4b50,590300bd-3daf-4d5e-9274-7a3782261f7e,...,None,None,None,None,None,None,None,None,None,None
1,2024-01-15,2024-01-15,CARD PAYMENT TO MGP*VINTED 82219278 ON 14-01-2024,OTT DEBIT,-16.40,None,booked,341d55c6-5bd9-354f-8330-955e86dc2a05,81becb568a4ca47239fdf2424de66807,590300bd-3daf-4d5e-9274-7a3782261f7e,...,None,None,None,None,None,None,None,None,None,None
2,2024-01-15,2024-01-15,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,-23.18,None,booked,03a7d23b-29b0-36b4-b23d-f271e1d87598,a8daa69bcd882741892ca4ae58c9d4dd,590300bd-3daf-4d5e-9274-7a3782261f7e,...,None,None,None,None,None,None,None,None,None,None
3,2024-01-15,2024-01-15,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,-3.48,None,booked,3cd9a88a-f30b-32ae-baf1-54afce479306,470724a969711fed6bec8a44a7273d52,590300bd-3daf-4d5e-9274-7a3782261f7e,...,None,None,None,None,None,None,None,None,None,None
4,2024-01-15,2024-01-15,STANDING ORDER VIA FASTER PAYMENT TO GOVUK REF...,BANK TRANSFER DEBIT,-60.00,None,booked,74b40017-542e-3097-a58b-c45d7d098de2,405fc76fb42ea66cf9bf5cd95255c244,590300bd-3daf-4d5e-9274-7a3782261f7e,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,2024-01-03,2024-01-03,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-1000.00,None,booked,fa6ee6a4-959f-3ba1-b81e-6639f42d23b4,eaa1304aa105c984df9dd5c06cd3b7eb,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,...,None,None,None,None,None,None,None,None,None,None
540,2023-12-20,2023-12-20,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-3000.00,None,booked,9a8325ec-147d-3633-959a-1fdc8c354e3f,e58e25db66dd77b52040b03cceb6fe6b,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,...,None,None,None,None,None,None,None,None,None,None
541,2023-12-02,2023-12-02,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-595.99,None,booked,f22ab8ac-53b2-32e2-9d20-27c6faac9b78,39ecea2d93aa015cb017705be2caf9ce,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,...,None,None,None,None,None,None,None,None,None,None
542,2023-11-17,2023-11-17,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-1604.82,None,booked,459f3f09-139b-32b9-998e-5f3e22eba36d,9f4de61a7487689bb0511df47f95d975,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,...,None,None,None,None,None,None,None,None,None,None


# Get historic


In [45]:
import pandas as pd


def import_historic_data():
    # read from csv
    path_transactions_historic = Path(
        "../data/historic/df_transactions_raw.csv"
    )
    transactions_df = pd.read_csv(path_transactions_historic)
    # camel to snake case, keeping right of '.'
    transactions_dict = transactions_df.to_dict(orient="records")
    transactions_dict = utils.clean_column_names(transactions_dict)
    # validate with dataclass
    
    return transactions_dict

In [58]:
path_transactions_historic = Path(
    "../data/historic/df_transactions_raw.csv"
)
transactions_df = pd.read_csv(path_transactions_historic)
transactions_df = transactions_df.dropna(axis=1)
transactions_df.columns = [utils.keep_text_right_of_dot(utils.to_snake_case_with_dots(key)) for key in transactions_df.columns]

In [59]:
transactions_df

,booking_date,booking_date_time,remittance_information_unstructured,proprietary_bank_transaction_code,amount,currency,status,account_id,account_type,account_name
0,2024-01-16,2024-01-16T00:00:00Z,"BANK GIRO CREDIT REF SATALIA NP COMPLET, .",BANK TRANSFER CREDIT,186.20,GBP,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
1,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO MGP*VINTED 82219278 ON 14-01-2024,OTT DEBIT,-16.40,GBP,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
2,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,-23.18,GBP,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
3,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,-3.48,GBP,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
4,2024-01-15,2024-01-15T00:00:00Z,STANDING ORDER VIA FASTER PAYMENT TO GOVUK REF...,BANK TRANSFER DEBIT,-60.00,GBP,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
...,...,...,...,...,...,...,...,...,...,...
539,2024-01-03,2024-01-03T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-1000.00,GBP,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
540,2023-12-20,2023-12-20T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-3000.00,GBP,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
541,2023-12-02,2023-12-02T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-595.99,GBP,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
542,2023-11-17,2023-11-17T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,-1604.82,GBP,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account


In [46]:
historic_transactions = import_historic_data()
historic_transactions_df = pd.DataFrame(historic_transactions).dropna(axis=1)

In [48]:
historic_transactions[0]

{'transaction_id': 'e770cf3d-b34c-32b7-acbd-2dbe7f94cdcd',
 'booking_date': '2024-01-16',
 'booking_date_time': '2024-01-16T00:00:00Z',
 'remittance_information_unstructured': 'BANK GIRO CREDIT REF SATALIA NP COMPLET, .',
 'proprietary_bank_transaction_code': 'BANK TRANSFER CREDIT',
 'internal_transaction_id': 'cfd6a5cecb851cd6f88f259dcfba4b50',
 'amount': nan,
 'currency': nan,
 'debtor_name': nan,
 'creditor_name': nan,
 'status': 'booked',
 'account_id': '590300bd-3daf-4d5e-9274-7a3782261f7e',
 'account_type': 'CACC',
 'account_name': 'joint account',
 'merchant_category_code': nan,
 'source_currency': nan,
 'exchange_rate': nan,
 'unit_currency': nan,
 'target_currency': nan,
 'quotation_date': nan,
 'value_date': nan,
 'value_date_time': nan}

In [42]:
latest_transactions_df = pd.DataFrame(latest_data.transactions).dropna(axis=1)
# combined_transactions_df = combine_transactions(historic_transactions_df,latest_transactions_df).reset_index(drop=True)


In [44]:
historic_transactions_df

,booking_date,booking_date_time,remittance_information_unstructured,proprietary_bank_transaction_code,status,account_id,account_type,account_name
0,2024-01-16,2024-01-16T00:00:00Z,"BANK GIRO CREDIT REF SATALIA NP COMPLET, .",BANK TRANSFER CREDIT,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
1,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO MGP*VINTED 82219278 ON 14-01-2024,OTT DEBIT,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
2,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
3,2024-01-15,2024-01-15T00:00:00Z,CARD PAYMENT TO PAYPAL *UBER EATS ON 12-01-2024,OTT DEBIT,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
4,2024-01-15,2024-01-15T00:00:00Z,STANDING ORDER VIA FASTER PAYMENT TO GOVUK REF...,BANK TRANSFER DEBIT,booked,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account
...,...,...,...,...,...,...,...,...
539,2024-01-03,2024-01-03T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
540,2023-12-20,2023-12-20T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
541,2023-12-02,2023-12-02T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account
542,2023-11-17,2023-11-17T00:00:00Z,CREDIT CARD PAYMENT,BANK TRANSFER DEBIT,booked,e9e5f8b9-da61-49ce-bdae-56546ce4a1c9,CACC,single account


In [38]:
latest_transactions_df.columns.sort_values()

Index(['account_id', 'account_name', 'account_type', 'amount', 'booking_date',
       'booking_date_time', 'currency', 'proprietary_bank_transaction_code',
       'remittance_information_unstructured', 'status'],
      dtype='object')

In [ ]:
# filtered_df = historic_transactions_df.loc[
#     (historic_transactions_df["amount"] > 0)
#     & (historic_transactions_df["flow"] == "credit")
# ]

In [20]:
def combine_transactions(historic_df: pd.DataFrame, df: pd.DataFrame) -> pd.DataFrame:
    logger.debug(f"joining historic and live transaction tables")

    # filter
    historic_df = historic_df.loc[historic_df["status"] == "booked"]
    
    # add source
    historic_df["source"]="historic"
    df["source"]="api"

    # combine
    combined_transactions_df = pd.concat([historic_df, df]).drop_duplicates()
    
    # clean
    combined_transactions_df.drop_duplicates().sort_values(
        by="booking_date",
    )
    combined_transactions_df = combined_transactions_df.sort_values(
        by="booking_date", ascending=False
    )
    combined_transactions_df = combined_transactions_df.drop_duplicates()
    combined_transactions_df = combined_transactions_df.reset_index(drop=True)
    return combined_transactions_df

In [ ]:
combined_transactions_df.head()

In [ ]:
combined_transactions = [model.Transaction(**d) for d in combined_transactions_df.to_dict(orient="records")]

In [ ]:
transactions = process_transactions.process_transactions(
    combined_transactions
)

In [ ]:
pd.DataFrame(transactions).head()

# Combine


# Tidy up output


In [ ]:
user_data = data_import.load_user_data()

In [ ]:
user_data.requisition.id

# Enrich


In [ ]:
user_data.transactions = process_transactions.process_transactions(
    user_data.transactions
)

# Query


In [ ]:
import pandas as pd

In [ ]:
records = user_data.query_transactions_by_transaction_code(
    "FASTER PAYMENT DEBIT REVERSAL"
)